## Producer Module
Note : This module is used to upload the events to the event hub

In [0]:
%pip install azure-identity azure-eventhub

In [0]:
%restart_python

In [0]:
# sanity check
import azure.identity, azure.eventhub
print("azure-identity:", azure.identity.__version__)
print("azure-eventhub:", azure.eventhub.__version__)

In [0]:
dbutils.secrets.list('gijoformula1-scope')

In [0]:
from azure.identity import ClientSecretCredential
from azure.eventhub import EventHubProducerClient, EventData

# --- secrets from Key Vault scope ---
SCOPE = "gijoformula1-scope"
tenant_id     = dbutils.secrets.get(scope=SCOPE, key="gijoformula1-app-tenent-id")
client_id     = dbutils.secrets.get(scope=SCOPE, key="gijoformula1-app-client-id")
client_secret = dbutils.secrets.get(scope=SCOPE, key="keh-client-secret")

cred = ClientSecretCredential(tenant_id, client_id, client_secret)
producer = EventHubProducerClient(
    fully_qualified_namespace="creditrisk.servicebus.windows.net",
    eventhub_name="customers",
    credential=cred
)
with producer:
    producer.send_batch([EventData('{"smoke":"ok"}')])
print("Sent 1 event")

In [0]:
bootstrap = "creditrisk.servicebus.windows.net:9093"
print(bootstrap)

In [0]:
# run once after installing azure-identity/azure-eventhub and restarting Python
from azure.identity import ClientSecretCredential
from azure.eventhub import EventHubProducerClient, EventData

SCOPE = "gijoformula1-scope"
tenant_id     = dbutils.secrets.get(SCOPE, "gijoformula1-app-tenent-id")
client_id     = dbutils.secrets.get(SCOPE, "gijoformula1-app-client-id")
client_secret = dbutils.secrets.get(SCOPE, "keh-client-secret")

cred = ClientSecretCredential(tenant_id, client_id, client_secret)

try:
    producer = EventHubProducerClient(
        fully_qualified_namespace="creditrisk.servicebus.windows.net",
        eventhub_name="customers",
        credential=cred,
    )
    with producer:
        producer.send_batch([EventData('{"probe":"ok"}')])
    print("✅ AMQP probe sent to 'customers'")
except Exception as e:
    print("❌ AMQP probe failed:", type(e).__name__, "-", str(e))


In [0]:
# If you haven't yet on this cluster:
# %pip install --upgrade azure-identity azure-eventhub
# dbutils.library.restartPython()

from azure.identity import ClientSecretCredential
from azure.eventhub import EventHubProducerClient, EventData

SCOPE = "gijoformula1-scope"
tenant_id     = dbutils.secrets.get(SCOPE, "gijoformula1-app-tenent-id")
client_id     = dbutils.secrets.get(SCOPE, "gijoformula1-app-client-id")
client_secret = dbutils.secrets.get(SCOPE, "keh-client-secret")

cred = ClientSecretCredential(tenant_id, client_id, client_secret)
NS   = "creditrisk.servicebus.windows.net"

def send_dbfs_jsonl_to_hub(dbfs_path: str, hub_name: str, batch_bytes: int = 256*1024):
    # Read file from DBFS with Spark (works on shared UC clusters)
    df = spark.read.text(dbfs_path).selectExpr("value")  # one line per Row(value=...)
    producer = EventHubProducerClient(
        fully_qualified_namespace=NS,
        eventhub_name=hub_name,
        credential=cred
    )
    sent = 0
    with producer:
        batch = producer.create_batch(max_size_in_bytes=batch_bytes)
        for row in df.toLocalIterator():          # stream lines to the driver
            line = row["value"]
            ev = EventData(line)
            try:
                batch.add(ev)
            except ValueError:
                producer.send_batch(batch)
                sent += len(batch)
                batch = producer.create_batch(max_size_in_bytes=batch_bytes)
                batch.add(ev)
        if len(batch) > 0:
            producer.send_batch(batch)
            sent += len(batch)
    print(f"Sent {sent} events to '{hub_name}' from {dbfs_path}")

send_dbfs_jsonl_to_hub("dbfs:/FileStore/tables/customers.jsonl", "customers")
send_dbfs_jsonl_to_hub("dbfs:/FileStore/tables/transactions.jsonl", "transactions")
send_dbfs_jsonl_to_hub("dbfs:/FileStore/tables/credit_applications.jsonl", "credit_applications")
